In [1]:
!pip install -r requirements.txt


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="dangvantuan/vietnamese-embedding",
    model_kwargs={"device": "cpu"},
)

c:\Users\toan.dn.nguyen\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\toan.dn.nguyen\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\toan.dn.nguyen\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

embeddings = HuggingFaceInstructEmbeddings(
        model_name="hkunlp/instructor-large",
        model_kwargs={"device": "cpu"},
)

TypeError: INSTRUCTOR._load_sbert_model() got an unexpected keyword argument 'token'

In [9]:
from chromadb.config import Settings
from langchain_community.vectorstores.chroma import Chroma

setting = Settings(
    anonymized_telemetry=False,
    is_persistent=True,
)

vector_store = Chroma(
    persist_directory='.DB_dangvantuan',
    embedding_function=embeddings,
    client_settings=setting,
)
retriever = vector_store.as_retriever()

In [10]:
result_docs = retriever.invoke("Số lượng khách quốc tế đến Việt Nam trong tháng 8/2024 là bao nhiêu và tăng trưởng so với cùng kỳ năm trước như thế nào?")

In [12]:
result_docs

[Document(page_content='Đến ngày thứ ba, hợp tử phát triển thành phôi dâu (morula) gồm 8 tế\nbào là các tế bào vạn năng, có thể biệt hóa thành toàn bộ phôi thai, bao gồm cả bánh nhau và\nthai nhi.'),
 Document(page_content='Khách bắt đầu\nđặt từ đầu tháng 8.'),
 Document(page_content='Phát ban sởi có thể kéo dài khoảng 7 ngày.'),
 Document(page_content='Đến tuần thứ 8 thai kỳ, hầu hết các cơ quan, bộ phận của phôi thai đã được hình\nthành ở mức cơ bản.')]

In [14]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
            """
            <s> [INST] Bạn là môt trợ lý ảo chuyên trả lời các câu hỏi liên quan đến lĩnh vực y tế và du lịch.\
            Đừng cung cấp cho tôi bất kỳ thông tin nào ngoài những thông tin được cung cấp. \
            Nếu bạn không biết câu trả lời, chỉ cần nói rằng bạn không biết hoặc chưa đủ thông tin. \
            Hãy trả lời toàn bộ bằng Tiếng Việt [/INST] </s> 
            [INST] Question: {question} 
            Context: {context} 
            Answer: [/INST]
            """
)

In [15]:
from langchain_community.chat_models import ChatOllama

model = ChatOllama(model="llama3")

In [16]:
from langchain.schema.runnable import RunnableConfig, RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [20]:
chain.invoke("Số lượng khách quốc tế đến Việt Nam trong tháng 8/2024 là bao nhiêu và tăng trưởng so với cùng kỳ năm trước như thế nào?")

"According to the Vietnam National Administration of Tourism (VNAT), the number of international tourists visiting Vietnam in August 2024 is estimated to be around 430,000.\n\nCompared to the same period last year, this represents a growth rate of approximately 12% due to the country's efforts to promote tourism and reopen its borders. The increase is expected to continue throughout the year, driven by factors such as improved air connectivity, enhanced infrastructure, and increased marketing efforts.\n\nPlease note that these numbers are subject to change based on various factors such as global travel trends, COVID-19 developments, and internal economic conditions.\n\nSource: Vietnam National Administration of Tourism (VNAT)"